<a href="https://colab.research.google.com/github/jackson-trader/CS462-Email-Spam-Classifier/blob/main/CS462_Email_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spam Likely | An Email Spam Classifier**

#### By Jackson Trader, Diego Lara, Mark Pack, & Alex Bryant
#### CS 46200 - Introduction To Artificial Intelligence

##### [Dataset from Kaggle](https://www.kaggle.com/datasets/ashfakyeafi/spam-email-classification)
##### [Link to GitHub repository](https://github.com/jackson-trader/CS462-Email-Spam-Classifier)

## **Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **Load Data**

ham = legitimate email <br>
spam = spam email

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jackson-trader/CS462-Email-Spam-Classifier/refs/heads/main/spam_data.csv') # DataFrame
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


## **Preprocess the Text**